## Personalized Learning (Localized Learning?)

#### This notebook includes the following online models;
1. A single global model with all data
2. Multiple local models (starting from a single global model)
   1. that are updated with new data
   2. that exchanges data in clusters
   3. that exchanges parameters in clusters

  
#### The dataset that is used for this project is [CIFAR-100 dataset][1]
* Has 100 classes containing 600 images each

#### New data are fed by the following rules;
1. Distributed, according to superclasses
  * Clusters will only be updated with data that belongs to a specific superclass
  * We update the NN by
    1. Changing all parameters of the NN
    2. Only changing the last few layers, as in many MTL models
2. Randomly (why?)

#### We expect to find an answer to the following research questions with this project;
1. If models are updated with data (or parameters) that are shared within a cluster, can the model perform good enough with the labels that count?
  * For example, the performance of the cluster that are updated with "Vehicles" superclass is only assessed with the labels that corresponds to the superclass.
  
[1]: https://www.cs.toronto.edu/~kriz/cifar.html

#### Questions

Retraining: how does it work <br>
How do we compare these models?


### Implementation with Custom Neural Network and MNIST dataset

In [5]:
%load_ext tensorboard

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines

In [6]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [8]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

#### Load MNIST dataset

In [9]:
# the data, split between train and test sets
(x_train, y_train_noncat), (x_test, y_test) = mnist.load_data()

In [10]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [11]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [126]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train_noncat, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
def custom_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [9]:
def compile_model(model):  
    # initiate SGD optimizer
    opt = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

In [10]:
def compile_model_lr(model, lr):  
    # initiate SGD optimizer
    opt = keras.optimizers.SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

#### We use 30000 samples to train global model and 30000 samples to simulate data being updated locally

In [11]:
global_sample_num = 30000

In [12]:
# it seems like the training data is already in random order
x_train_global = x_train[:global_sample_num]
x_train_local = x_train[global_sample_num:]
y_train_global = y_train[:global_sample_num]
y_train_local = y_train[global_sample_num:]

In [13]:
def split_local_data(num):
    # if unable to split into same number of datasets,
    # drop last few data
    last_index = 30000 - (30000 % num) 
    x_local = np.split(x_train_local[:last_index], num)
    y_local = np.split(y_train_local[:last_index], num)
    return x_local, y_local

In [7]:
def fit_model_global(model, epochs):
    now = datetime.datetime.now()
    print ("Training date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    return model.fit(x_train_global, y_train_global,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_data=(x_test, y_test_cat),
                      shuffle=True, callbacks=[tensorboard_callback])


#### A few helper functions

In [15]:
def compile_model_rms(model):  
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [16]:
def print_datetime():
    now = datetime.datetime.now()
    print ("Training date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))

### Test with global model : Tune parameters

In [12]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0804 23:44:36.949888 140148921943808 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 14s 229us/step - loss: 0.0853 - acc: 0.2947 - val_loss: 0.0623 - val_acc: 0.6987
Epoch 2/12
60000/60000 [==============================] - 6s 93us/step - loss: 0.0381 - acc: 0.7438 - val_loss: 0.0168 - val_acc: 0.8886
Epoch 3/12
60000/60000 [==============================] - 6s 93us/step - loss: 0.0231 - acc: 0.8447 - val_loss: 0.0128 - val_acc: 0.9166
Epoch 4/12
60000/60000 [==============================] - 6s 93us/step - loss: 0.0196 - acc: 0.8698 - val_loss: 0.0111 - val_acc: 0.9283
Epoch 5/12
60000/60000 [==============================] - 6s 93us/step - loss: 0.0175 - acc: 0.8845 - val_loss: 0.0100 - val_acc: 0.9355
Epoch 6/12
60000/60000 [==============================] - 6s 94us/step - loss: 0.0161 - acc: 0.8932 - val_loss: 0.0092 - val_acc: 0.9404
Epoch 7/12
60000/60000 [==============================] - 6s 93us/step - loss: 0.0151 - acc: 0.9006 - val_loss: 0.0085 - val_a

## RQ1: Serial vs. Parallel

#### Serial
30k - 10k - 10k - 10k 


In [25]:
print_datetime()

serial_model = custom_model()
compile_model(serial_model)
history_serial = serial_model.fit(x_train_global, y_train_global,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = serial_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0805 00:37:06.503879 139951117633280 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 00:37:06.522261 139951117633280 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 00:37:06.524887 139951117633280 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 00:37:06.552422 139951117633280 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The 

Training date and time : 
2019-08-05 00:37:06


W0805 00:37:06.730019 139951117633280 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0805 00:37:06.921309 139951117633280 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 30000 samples, validate on 10000 samples
Epoch 1/12
30000/30000 [==============================] - 5s 180us/step - loss: 0.0889 - acc: 0.1921 - val_loss: 0.0867 - val_acc: 0.3592
Epoch 2/12
30000/30000 [==============================] - 3s 98us/step - loss: 0.0787 - acc: 0.4138 - val_loss: 0.0551 - val_acc: 0.6575
Epoch 3/12
30000/30000 [==============================] - 3s 97us/step - loss: 0.0445 - acc: 0.6927 - val_loss: 0.0223 - val_acc: 0.8654
Epoch 4/12
30000/30000 [==============================] - 3s 99us/step - loss: 0.0290 - acc: 0.8046 - val_loss: 0.0168 - val_acc: 0.8944
Epoch 5/12
30000/30000 [==============================] - 3s 97us/step - loss: 0.0241 - acc: 0.8386 - val_loss: 0.0146 - val_acc: 0.9059
Epoch 6/12
30000/30000 [==============================] - 3s 99us/step - loss: 0.0217 - acc: 0.8547 - val_loss: 0.0131 - val_acc: 0.9139
Epoch 7/12
30000/30000 [==============================] - 3s 97us/step - loss: 0.0201 - acc: 0.8659 - val_loss: 0.0123 - val_ac

In [28]:
history_serial.history['val_acc'][-1]

0.9399

In [29]:
serial_model.save("30k_pretrain.h5")

In [44]:
serial_model = keras.models.load_model('30k_pretrain.h5')

In [45]:
len(serial_model.layers)

8

In [48]:
local_serial_hist = []
split_num = 10

(x_local, y_local) = split_local_data(split_num)

In [49]:
for i in range(split_num):
    print("Local training #%d" %(i+1))
    serial_model.fit(x_local[i], y_local[i],
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

Local training #1
Train on 3000 samples, validate on 10000 samples
Epoch 1/12
3000/3000 [==============================] - 1s 387us/step - loss: 0.0182 - acc: 0.8777 - val_loss: 0.0096 - val_acc: 0.9365
Epoch 2/12
3000/3000 [==============================] - 1s 192us/step - loss: 0.0177 - acc: 0.8830 - val_loss: 0.0095 - val_acc: 0.9376
Epoch 3/12
3000/3000 [==============================] - 1s 179us/step - loss: 0.0181 - acc: 0.8790 - val_loss: 0.0095 - val_acc: 0.9395
Epoch 4/12
3000/3000 [==============================] - 1s 178us/step - loss: 0.0174 - acc: 0.8850 - val_loss: 0.0094 - val_acc: 0.9386
Epoch 5/12
3000/3000 [==============================] - 1s 178us/step - loss: 0.0173 - acc: 0.8857 - val_loss: 0.0093 - val_acc: 0.9387
Epoch 6/12
3000/3000 [==============================] - 1s 184us/step - loss: 0.0167 - acc: 0.8887 - val_loss: 0.0093 - val_acc: 0.9386
Epoch 7/12
3000/3000 [==============================] - 1s 183us/step - loss: 0.0175 - acc: 0.8843 - val_loss: 0.0093

Epoch 11/12
3000/3000 [==============================] - 1s 180us/step - loss: 0.0128 - acc: 0.9147 - val_loss: 0.0083 - val_acc: 0.9454
Epoch 12/12
3000/3000 [==============================] - 1s 182us/step - loss: 0.0131 - acc: 0.9180 - val_loss: 0.0084 - val_acc: 0.9461
Local training #6
Train on 3000 samples, validate on 10000 samples
Epoch 1/12
3000/3000 [==============================] - 1s 180us/step - loss: 0.0162 - acc: 0.8923 - val_loss: 0.0082 - val_acc: 0.9467
Epoch 2/12
3000/3000 [==============================] - 1s 179us/step - loss: 0.0155 - acc: 0.8970 - val_loss: 0.0082 - val_acc: 0.9476
Epoch 3/12
3000/3000 [==============================] - 1s 178us/step - loss: 0.0159 - acc: 0.8957 - val_loss: 0.0082 - val_acc: 0.9477
Epoch 4/12
3000/3000 [==============================] - 1s 178us/step - loss: 0.0149 - acc: 0.9050 - val_loss: 0.0082 - val_acc: 0.9474
Epoch 5/12
3000/3000 [==============================] - 1s 178us/step - loss: 0.0148 - acc: 0.8993 - val_loss: 0.00

In [57]:
serial_model.save("serial_split_10.h5")

#### Parallel
<pre>
       | 10k | 
30k -> | 10k | -> res
       | 10k | 
</pre>

In [50]:
parallel_model = []
for i in range(split_num):
    parallel_model.append(keras.models.load_model('30k_pretrain.h5')

In [51]:
for i in range(split_num):
    print("Local training (parallel) #%d" %(i+1))
    parallel_model[i].fit(x_local[i], y_local[i],
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

Local training (parallel) #1
Train on 3000 samples, validate on 10000 samples
Epoch 1/12
3000/3000 [==============================] - 1s 471us/step - loss: 0.0182 - acc: 0.8723 - val_loss: 0.0095 - val_acc: 0.9381
Epoch 2/12
3000/3000 [==============================] - 1s 194us/step - loss: 0.0177 - acc: 0.8867 - val_loss: 0.0095 - val_acc: 0.9380
Epoch 3/12
3000/3000 [==============================] - 1s 184us/step - loss: 0.0178 - acc: 0.8853 - val_loss: 0.0095 - val_acc: 0.9390
Epoch 4/12
3000/3000 [==============================] - 1s 182us/step - loss: 0.0170 - acc: 0.8927 - val_loss: 0.0095 - val_acc: 0.9372
Epoch 5/12
3000/3000 [==============================] - 1s 184us/step - loss: 0.0173 - acc: 0.8850 - val_loss: 0.0094 - val_acc: 0.9370
Epoch 6/12
3000/3000 [==============================] - 1s 183us/step - loss: 0.0169 - acc: 0.8907 - val_loss: 0.0094 - val_acc: 0.9377
Epoch 7/12
3000/3000 [==============================] - 1s 183us/step - loss: 0.0165 - acc: 0.8900 - val_l

3000/3000 [==============================] - 1s 184us/step - loss: 0.0109 - acc: 0.9253 - val_loss: 0.0096 - val_acc: 0.9358
Epoch 8/12
3000/3000 [==============================] - 1s 184us/step - loss: 0.0109 - acc: 0.9327 - val_loss: 0.0097 - val_acc: 0.9361
Epoch 9/12
3000/3000 [==============================] - 1s 185us/step - loss: 0.0105 - acc: 0.9333 - val_loss: 0.0096 - val_acc: 0.9365
Epoch 10/12
3000/3000 [==============================] - 1s 185us/step - loss: 0.0113 - acc: 0.9260 - val_loss: 0.0096 - val_acc: 0.9354
Epoch 11/12
3000/3000 [==============================] - 1s 185us/step - loss: 0.0108 - acc: 0.9317 - val_loss: 0.0097 - val_acc: 0.9361
Epoch 12/12
3000/3000 [==============================] - 1s 186us/step - loss: 0.0103 - acc: 0.9333 - val_loss: 0.0097 - val_acc: 0.9360


In [52]:
weights = [m.get_weights() for m in parallel_model]

In [53]:
# Federated Aggregation
new_weights = list()

for weights_list_tuple in zip(*weights):
    new_weights.append(
        [np.array(weights_).mean(axis=0)\
            for weights_ in zip(*weights_list_tuple)])

In [54]:
pretrained_model = keras.models.load_model('30k_pretrain.h5')
aggr_model = keras.models.clone_model(pretrained_model)
aggr_model.set_weights(new_weights)

partial aggregation

In [55]:
compile_model(aggr_model)
aggr_model.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 172us/step


[0.009055071101541398, 0.9402]

In [56]:
aggr_model.save("parallel_split_10.h5")

serial vs. parallel by devices

In [22]:
def compare_by_device(lower, upper):
    # Hyperparameter
    lr = 0.001
    
    serial_res = np.zeros(upper - lower + 1)
    parallel_res = np.zeros(upper - lower + 1)
    serial_res[0] = 0.9563
    parallel_res[0] = 0.9563
    for split_num in range(lower, upper):
        print("<=======   Learning with %d devices   =======>" %split_num)
        start_time = datetime.datetime.now() 
        
        #split dataset
        (x_local, y_local) = split_local_data(split_num)
        
        # run serial model
        serial = keras.models.load_model('30k_pretrain.h5')
        for layer in serial.layers[:2]:
            layer.trainable=False
        for layer in serial.layers[2:]:
            layer.trainable=True
        
        compile_model_lr(serial, lr)
        
        for i in range(split_num):
            print("%d devices: Local training (serial) #%d" %(split_num, i+1))
            serial_hist = serial.fit(x_local[i], y_local[i],
                              batch_size=batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
        serial_res[split_num - lower + 1] = serial_hist.history['val_acc'][-1]
#         serial.save("bd_serial_split_" + str(split_num) + ".h5")
        
        # run parallel model
        parallel = [keras.models.load_model('30k_pretrain.h5') for _ in range(split_num)]
        for p in parallel:
            for layer in p.layers[:2]:
                layer.trainable=False
            for layer in p.layers[2:]:
                layer.trainable=True
            compile_model_lr(p, lr)
            
        for i in range(split_num):
            print("%d devices: Local training (parallel) #%d" %(split_num, i+1))
            parallel[i].fit(x_local[i], y_local[i],
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(x_test, y_test))
        weights = [m.get_weights() for m in parallel]
        # Federated Aggregation
        new_weights = list()
        for weights_list_tuple in zip(*weights):
            new_weights.append(
                [np.array(weights_).mean(axis=0)\
                    for weights_ in zip(*weights_list_tuple)])
            
        pretrained_model = keras.models.load_model('30k_pretrain.h5')
        aggr_model = keras.models.clone_model(pretrained_model)
        aggr_model.set_weights(new_weights)
        compile_model(aggr_model)
        aggr_score = aggr_model.evaluate(x_test, y_test)
#         aggr_model.save("bd_parallel_split_" + str(split_num) + ".h5")
        parallel_res[split_num - lower + 1] = aggr_score[1]
        
        end_time = datetime.datetime.now() 
        elasped_time = end_time - start_time
        et = divmod(elasped_time.days * 86400 + elasped_time.seconds, 60)
        print("Elasped Time for %d devices: %d minutes, %d seconds" %(split_num, et[0], et[1]))
    return serial_res, parallel_res

In [ ]:
lower = 2
upper = 12
(sr_2, pr_2) = compare_by_device(lower,upper)

<=======   Learning with 2 devices   =======>
2 devices: Local training (serial) #1
Train on 15000 samples, validate on 10000 samples
Epoch 1/12
15000/15000 [==============================] - 13s 836us/step - loss: 0.0161 - acc: 0.8938 - val_loss: 0.0092 - val_acc: 0.9419
Epoch 2/12
15000/15000 [==============================] - 1s 84us/step - loss: 0.0160 - acc: 0.8954 - val_loss: 0.0092 - val_acc: 0.9415
Epoch 3/12
15000/15000 [==============================] - 1s 84us/step - loss: 0.0154 - acc: 0.9009 - val_loss: 0.0092 - val_acc: 0.9413
Epoch 4/12
15000/15000 [==============================] - 1s 85us/step - loss: 0.0156 - acc: 0.8993 - val_loss: 0.0092 - val_acc: 0.9409
Epoch 5/12
15000/15000 [==============================] - 1s 84us/step - loss: 0.0156 - acc: 0.8963 - val_loss: 0.0091 - val_acc: 0.9418
Epoch 6/12
15000/15000 [==============================] - 1s 84us/step - loss: 0.0155 - acc: 0.8995 - val_loss: 0.0091 - val_acc: 0.9409
Epoch 7/12
15000/15000 [==================

Epoch 8/12
10000/10000 [==============================] - 1s 98us/step - loss: 0.0154 - acc: 0.8997 - val_loss: 0.0091 - val_acc: 0.9412
Epoch 9/12
10000/10000 [==============================] - 1s 98us/step - loss: 0.0160 - acc: 0.8950 - val_loss: 0.0091 - val_acc: 0.9420
Epoch 10/12
10000/10000 [==============================] - 1s 98us/step - loss: 0.0161 - acc: 0.8938 - val_loss: 0.0091 - val_acc: 0.9420
Epoch 11/12
10000/10000 [==============================] - 1s 100us/step - loss: 0.0157 - acc: 0.8965 - val_loss: 0.0090 - val_acc: 0.9421
Epoch 12/12
10000/10000 [==============================] - 1s 97us/step - loss: 0.0153 - acc: 0.8985 - val_loss: 0.0091 - val_acc: 0.9419
3 devices: Local training (serial) #2
Train on 10000 samples, validate on 10000 samples
Epoch 1/12
10000/10000 [==============================] - 1s 97us/step - loss: 0.0161 - acc: 0.8923 - val_loss: 0.0090 - val_acc: 0.9418
Epoch 2/12
10000/10000 [==============================] - 1s 97us/step - loss: 0.0159 

7500/7500 [==============================] - 1s 111us/step - loss: 0.0124 - acc: 0.9191 - val_loss: 0.0088 - val_acc: 0.9431
4 devices: Local training (parallel) #1
Train on 7500 samples, validate on 10000 samples
Epoch 1/12
7500/7500 [==============================] - 14s 2ms/step - loss: 0.0164 - acc: 0.8927 - val_loss: 0.0093 - val_acc: 0.9412
Epoch 2/12
7500/7500 [==============================] - 1s 114us/step - loss: 0.0161 - acc: 0.8957 - val_loss: 0.0092 - val_acc: 0.9415
Epoch 3/12
7500/7500 [==============================] - 1s 115us/step - loss: 0.0160 - acc: 0.8939 - val_loss: 0.0092 - val_acc: 0.9410
Epoch 4/12
7500/7500 [==============================] - 1s 114us/step - loss: 0.0157 - acc: 0.8963 - val_loss: 0.0092 - val_acc: 0.9413
Epoch 5/12
7500/7500 [==============================] - 1s 114us/step - loss: 0.0159 - acc: 0.8951 - val_loss: 0.0092 - val_acc: 0.9415
Epoch 6/12
7500/7500 [==============================] - 1s 114us/step - loss: 0.0163 - acc: 0.8933 - val_lo

Epoch 8/12
6000/6000 [==============================] - 1s 129us/step - loss: 0.0155 - acc: 0.9000 - val_loss: 0.0092 - val_acc: 0.9409
Epoch 9/12
6000/6000 [==============================] - 1s 128us/step - loss: 0.0155 - acc: 0.8990 - val_loss: 0.0092 - val_acc: 0.9410
Epoch 10/12
6000/6000 [==============================] - 1s 129us/step - loss: 0.0155 - acc: 0.8957 - val_loss: 0.0092 - val_acc: 0.9412
Epoch 11/12
6000/6000 [==============================] - 1s 127us/step - loss: 0.0160 - acc: 0.8948 - val_loss: 0.0092 - val_acc: 0.9415
Epoch 12/12
6000/6000 [==============================] - 1s 126us/step - loss: 0.0156 - acc: 0.8987 - val_loss: 0.0092 - val_acc: 0.9413
5 devices: Local training (serial) #2
Train on 6000 samples, validate on 10000 samples
Epoch 1/12
6000/6000 [==============================] - 1s 127us/step - loss: 0.0156 - acc: 0.8978 - val_loss: 0.0091 - val_acc: 0.9412
Epoch 2/12
6000/6000 [==============================] - 1s 128us/step - loss: 0.0156 - acc: 0.

Epoch 5/12
6000/6000 [==============================] - 1s 131us/step - loss: 0.0160 - acc: 0.8943 - val_loss: 0.0093 - val_acc: 0.9402
Epoch 6/12
6000/6000 [==============================] - 1s 130us/step - loss: 0.0158 - acc: 0.8970 - val_loss: 0.0092 - val_acc: 0.9399
Epoch 7/12
6000/6000 [==============================] - 1s 130us/step - loss: 0.0162 - acc: 0.8942 - val_loss: 0.0092 - val_acc: 0.9408
Epoch 8/12
6000/6000 [==============================] - 1s 130us/step - loss: 0.0160 - acc: 0.8942 - val_loss: 0.0092 - val_acc: 0.9397
Epoch 9/12
6000/6000 [==============================] - 1s 129us/step - loss: 0.0161 - acc: 0.8937 - val_loss: 0.0092 - val_acc: 0.9407
Epoch 10/12
6000/6000 [==============================] - 1s 130us/step - loss: 0.0157 - acc: 0.8965 - val_loss: 0.0092 - val_acc: 0.9405
Epoch 11/12
6000/6000 [==============================] - 1s 129us/step - loss: 0.0155 - acc: 0.8980 - val_loss: 0.0092 - val_acc: 0.9410
Epoch 12/12
6000/6000 [=======================

...

Epoch 6/12
3000/3000 [==============================] - 1s 222us/step - loss: 0.0141 - acc: 0.9093 - val_loss: 0.0092 - val_acc: 0.9405
Epoch 7/12
3000/3000 [==============================] - 1s 222us/step - loss: 0.0138 - acc: 0.9097 - val_loss: 0.0093 - val_acc: 0.9400
Epoch 8/12
3000/3000 [==============================] - 1s 222us/step - loss: 0.0139 - acc: 0.9083 - val_loss: 0.0092 - val_acc: 0.9402
Epoch 9/12
3000/3000 [==============================] - 1s 222us/step - loss: 0.0138 - acc: 0.9113 - val_loss: 0.0092 - val_acc: 0.9404
Epoch 10/12
3000/3000 [==============================] - 1s 220us/step - loss: 0.0145 - acc: 0.9037 - val_loss: 0.0092 - val_acc: 0.9403
Epoch 11/12
3000/3000 [==============================] - 1s 221us/step - loss: 0.0139 - acc: 0.9080 - val_loss: 0.0093 - val_acc: 0.9401
Epoch 12/12
3000/3000 [==============================] - 1s 220us/step - loss: 0.0135 - acc: 0.9123 - val_loss: 0.0093 - val_acc: 0.9403
10 devices: Local training (serial) #3
Train 

Epoch 3/12
3000/3000 [==============================] - 1s 223us/step - loss: 0.0165 - acc: 0.8893 - val_loss: 0.0089 - val_acc: 0.9428
Epoch 4/12
3000/3000 [==============================] - 1s 222us/step - loss: 0.0158 - acc: 0.8980 - val_loss: 0.0089 - val_acc: 0.9432
Epoch 5/12
3000/3000 [==============================] - 1s 221us/step - loss: 0.0157 - acc: 0.8947 - val_loss: 0.0089 - val_acc: 0.9428
Epoch 6/12
3000/3000 [==============================] - 1s 221us/step - loss: 0.0155 - acc: 0.8977 - val_loss: 0.0089 - val_acc: 0.9426
Epoch 7/12
3000/3000 [==============================] - 1s 225us/step - loss: 0.0159 - acc: 0.8943 - val_loss: 0.0089 - val_acc: 0.9430
Epoch 8/12
3000/3000 [==============================] - 1s 221us/step - loss: 0.0161 - acc: 0.8947 - val_loss: 0.0089 - val_acc: 0.9428
Epoch 9/12
3000/3000 [==============================] - 1s 221us/step - loss: 0.0160 - acc: 0.8947 - val_loss: 0.0089 - val_acc: 0.9426
Epoch 10/12
3000/3000 [=========================

Epoch 9/12
3000/3000 [==============================] - 1s 230us/step - loss: 0.0168 - acc: 0.8867 - val_loss: 0.0092 - val_acc: 0.9418
Epoch 10/12
3000/3000 [==============================] - 1s 232us/step - loss: 0.0166 - acc: 0.8907 - val_loss: 0.0092 - val_acc: 0.9416
Epoch 11/12
3000/3000 [==============================] - 1s 228us/step - loss: 0.0165 - acc: 0.8907 - val_loss: 0.0092 - val_acc: 0.9416
Epoch 12/12
3000/3000 [==============================] - 1s 226us/step - loss: 0.0168 - acc: 0.8867 - val_loss: 0.0092 - val_acc: 0.9412
10 devices: Local training (parallel) #7
Train on 3000 samples, validate on 10000 samples
Epoch 1/12
3000/3000 [==============================] - 22s 7ms/step - loss: 0.0172 - acc: 0.8893 - val_loss: 0.0093 - val_acc: 0.9408
Epoch 2/12
3000/3000 [==============================] - 1s 230us/step - loss: 0.0170 - acc: 0.8870 - val_loss: 0.0093 - val_acc: 0.9407
Epoch 3/12
3000/3000 [==============================] - 1s 226us/step - loss: 0.0174 - acc: 

Epoch 5/12
2727/2727 [==============================] - 1s 249us/step - loss: 0.0178 - acc: 0.8816 - val_loss: 0.0092 - val_acc: 0.9419
Epoch 6/12
2727/2727 [==============================] - 1s 247us/step - loss: 0.0175 - acc: 0.8845 - val_loss: 0.0092 - val_acc: 0.9420
Epoch 7/12
2727/2727 [==============================] - 1s 248us/step - loss: 0.0176 - acc: 0.8856 - val_loss: 0.0092 - val_acc: 0.9417
Epoch 8/12
2727/2727 [==============================] - 1s 247us/step - loss: 0.0180 - acc: 0.8797 - val_loss: 0.0092 - val_acc: 0.9421
Epoch 9/12
2727/2727 [==============================] - 1s 248us/step - loss: 0.0182 - acc: 0.8750 - val_loss: 0.0092 - val_acc: 0.9420
Epoch 10/12
2727/2727 [==============================] - 1s 249us/step - loss: 0.0173 - acc: 0.8797 - val_loss: 0.0092 - val_acc: 0.9415
Epoch 11/12
2727/2727 [==============================] - 1s 248us/step - loss: 0.0181 - acc: 0.8764 - val_loss: 0.0092 - val_acc: 0.9416
Epoch 12/12
2727/2727 [=======================

Epoch 2/12
2727/2727 [==============================] - 1s 248us/step - loss: 0.0164 - acc: 0.8933 - val_loss: 0.0090 - val_acc: 0.9429
Epoch 3/12
2727/2727 [==============================] - 1s 246us/step - loss: 0.0161 - acc: 0.8973 - val_loss: 0.0090 - val_acc: 0.9426
Epoch 4/12
2727/2727 [==============================] - 1s 247us/step - loss: 0.0160 - acc: 0.8955 - val_loss: 0.0090 - val_acc: 0.9424
Epoch 5/12
2727/2727 [==============================] - 1s 247us/step - loss: 0.0161 - acc: 0.8981 - val_loss: 0.0090 - val_acc: 0.9431
Epoch 6/12
2727/2727 [==============================] - 1s 248us/step - loss: 0.0161 - acc: 0.8959 - val_loss: 0.0090 - val_acc: 0.9432
Epoch 7/12
2727/2727 [==============================] - 1s 248us/step - loss: 0.0169 - acc: 0.8911 - val_loss: 0.0090 - val_acc: 0.9432
Epoch 8/12
2727/2727 [==============================] - 1s 248us/step - loss: 0.0170 - acc: 0.8896 - val_loss: 0.0089 - val_acc: 0.9430
Epoch 9/12
2727/2727 [==========================

In [ ]:
x = np.arange(1,12)
sns.set_style("darkgrid")
plt.axis([0.5, 11, 0.9380, 0.96])
plt.plot(x, sr_2, label='serial')
plt.plot(x, pr_2, label='parallel')
# line for 
l = mlines.Line2D([-0.5,11], [0.9399,0.9399], c='red', label='base')
ax = plt.gca()
ax.add_line(l)
ax.legend()
plt.suptitle('SGD, Fixed, Learning rate: ' + str(0.001))
plt.xlabel('num_devices')
plt.ylabel('validation accuracy')
plt.savefig('SGD_lr001_fixed.png')
plt.show()


### Distributed Model 
#### 90% one class, 10% noise (non-iid)

In [17]:
num_users = 10

In [34]:
y_train_local = y_train_noncat[30000:]

In [35]:
y_train_local.shape

(30000,)

In [36]:
x_train_skewed = x_train_local[:27000]
x_train_noise = x_train_local[27000:]
y_train_skewed = y_train_local[:27000]
y_train_noise = y_train_local[27000:]

In [106]:
# make placeholders for datas
x_data_split = [np.empty([0,28,28,1]) for _ in range(num_users)]
y_data_split = [np.empty([0], dtype='uint8') for _ in range(num_users)]

In [109]:
# split data based on class
for i in range(y_train_skewed.shape[0]):
    if (i%1000 == 0):
        print("%d complete" %i)
    user_num = (int) (y_train_skewed[i]) # user_num is from 0 to 9, depends on class
    x_data_split[user_num] = \
        np.concatenate((x_data_split[user_num], np.expand_dims(x_train_skewed[i], axis=0)), axis=0)
    y_data_split[user_num] = \
        np.concatenate((y_data_split[user_num], np.expand_dims(y_train_skewed[i], axis=0)), axis=0)

0 complete
1000 complete
2000 complete
3000 complete
4000 complete
5000 complete
6000 complete
7000 complete
8000 complete
9000 complete
10000 complete
11000 complete
12000 complete
13000 complete
14000 complete
15000 complete
16000 complete
17000 complete
18000 complete
19000 complete
20000 complete
21000 complete
22000 complete
23000 complete
24000 complete
25000 complete
26000 complete


In [111]:
x_noise_split = np.split(x_train_noise, num_users)
y_noise_split = np.split(y_train_noise, num_users)

In [112]:
for i in range(num_users):
    x_data_split[i] = np.concatenate((x_data_split[i], x_noise_split[i]), axis=0)
    y_data_split[i] = np.concatenate((y_data_split[i], y_noise_split[i]), axis=0)

In [113]:
for i in range(10):
    print(x_data_split[i].shape)

(2965, 28, 28, 1)
(3294, 28, 28, 1)
(3001, 28, 28, 1)
(3044, 28, 28, 1)
(2926, 28, 28, 1)
(2750, 28, 28, 1)
(2955, 28, 28, 1)
(3126, 28, 28, 1)
(2979, 28, 28, 1)
(2960, 28, 28, 1)


In [115]:
y_data_split_cat = []
for yd in y_data_split:
    y_data_split_cat.append(keras.utils.to_categorical(yd, 10))

In [119]:
parallel_model = []
for i in range(10):
    parallel_model.append(keras.models.load_model('30k_pretrain.h5'))

W0813 04:12:18.179871 140186844686080 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 04:12:18.219593 140186844686080 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 04:12:18.251524 140186844686080 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0813 04:12:18.255886 140186844686080 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default

In [127]:
for i in range(10):
    print("Local training (parallel) #%d" %(i+1))
    parallel_model[i].fit(x_data_split[i], y_data_split_cat[i],
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

Local training (parallel) #1
Train on 2965 samples, validate on 10000 samples
Epoch 1/12
2965/2965 [==============================] - 5s 2ms/step - loss: 0.0043 - acc: 0.9710 - val_loss: 0.0138 - val_acc: 0.9096
Epoch 2/12
2965/2965 [==============================] - 1s 193us/step - loss: 0.0022 - acc: 0.9875 - val_loss: 0.0162 - val_acc: 0.8941
Epoch 3/12
2965/2965 [==============================] - 1s 188us/step - loss: 0.0023 - acc: 0.9848 - val_loss: 0.0152 - val_acc: 0.9002
Epoch 4/12
2965/2965 [==============================] - 1s 185us/step - loss: 0.0026 - acc: 0.9841 - val_loss: 0.0152 - val_acc: 0.8986
Epoch 5/12
2965/2965 [==============================] - 1s 188us/step - loss: 0.0022 - acc: 0.9858 - val_loss: 0.0150 - val_acc: 0.9006
Epoch 6/12
2965/2965 [==============================] - 1s 186us/step - loss: 0.0023 - acc: 0.9848 - val_loss: 0.0157 - val_acc: 0.8972
Epoch 7/12
2965/2965 [==============================] - 1s 186us/step - loss: 0.0025 - acc: 0.9838 - val_los

2960/2960 [==============================] - 1s 196us/step - loss: 0.0033 - acc: 0.9811 - val_loss: 0.0366 - val_acc: 0.7648
Epoch 8/12
2960/2960 [==============================] - 1s 197us/step - loss: 0.0034 - acc: 0.9780 - val_loss: 0.0360 - val_acc: 0.7710
Epoch 9/12
2960/2960 [==============================] - 1s 197us/step - loss: 0.0033 - acc: 0.9791 - val_loss: 0.0357 - val_acc: 0.7725
Epoch 10/12
2960/2960 [==============================] - 1s 194us/step - loss: 0.0032 - acc: 0.9814 - val_loss: 0.0355 - val_acc: 0.7755
Epoch 11/12
2960/2960 [==============================] - 1s 194us/step - loss: 0.0034 - acc: 0.9780 - val_loss: 0.0367 - val_acc: 0.7693
Epoch 12/12
2960/2960 [==============================] - 1s 195us/step - loss: 0.0029 - acc: 0.9834 - val_loss: 0.0356 - val_acc: 0.7764


The models that are fed with certain class more than others will distinguish that class with others better

when we are showing 1 class more than the others, are we just increasing tendency of the model to classify as that class, or are we actually helping it to learn more?

confusion matrix, 